In [2]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [3]:
train_data = pd.read_csv("Documents//train.csv")
test_data = pd.read_csv("Documents//test.csv")

In [4]:
train_data.tail()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900
188532,188532,Porsche,Macan Base,2016,59500,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,None reported,Yes,28995


In [5]:
test_data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [6]:
train_data.isnull().sum()

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [7]:
train_data.fillna(method='ffill', inplace=True)
test_data.fillna(method='ffill', inplace=True)


C:\Users\welcome\AppData\Local\Temp\ipykernel_18484\2079992739.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_data.fillna(method='ffill', inplace=True)
C:\Users\welcome\AppData\Local\Temp\ipykernel_18484\2079992739.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_data.fillna(method='ffill', inplace=True)


In [8]:
train_data.isnull().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

In [9]:


# Function to extract numeric values from the 'engine' feature
def extract_engine_features(engine_str):
    hp = re.search(r'(\d+(\.\d+)?)HP', engine_str)  # Extract Horsepower (HP)
    liters = re.search(r'(\d+(\.\d+)?)L', engine_str)  # Extract Liters (L)
    cylinders = re.search(r'(\d+) Cylinder', engine_str)  # Extract Cylinder count
    
    # If a match is found, return the value as a float, else return None
    hp_value = float(hp.group(1)) if hp else None
    liters_value = float(liters.group(1)) if liters else None
    cylinders_value = int(cylinders.group(1)) if cylinders else None
    
    return pd.Series([hp_value, liters_value, cylinders_value])

# Apply the function to the engine column
train_data[['HP', 'Liters', 'Cylinders']] = train_data['engine'].apply(extract_engine_features)
test_data[['HP', 'Liters', 'Cylinders']] = test_data['engine'].apply(extract_engine_features)

# Drop the original engine column since it's now converted to numeric form
train_data = train_data.drop(columns=['engine'])
test_data = test_data.drop(columns=['engine'])


In [10]:
train_data.head()

,id,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,HP,Liters,Cylinders
0,0,MINI,Cooper S Base,2007,213000,Gasoline,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0
1,1,Lincoln,LS V8,2002,143250,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9,8.0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3,8.0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0,8.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0,4.0


In [11]:
# Create a new feature: car_age
train_data['car_age'] = 2024 - train_data['model_year']
test_data['car_age'] = 2024 - test_data['model_year']
train_data = train_data.drop(columns=['model_year'])
test_data = test_data.drop(columns=['model_year'])


In [12]:
train_data.head()

,id,brand,model,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,HP,Liters,Cylinders,car_age
0,0,MINI,Cooper S Base,213000,Gasoline,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0,17
1,1,Lincoln,LS V8,143250,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9,8.0,22
2,2,Chevrolet,Silverado 2500 LT,136731,E85 Flex Fuel,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3,8.0,22
3,3,Genesis,G90 5.0 Ultimate,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0,8.0,7
4,4,Mercedes-Benz,Metris Base,7388,Gasoline,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0,4.0,3


In [13]:
test_data.head()

,id,brand,model,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,HP,Liters,Cylinders,car_age
0,188533,Land,Rover LR2 Base,98000,Gasoline,6-Speed A/T,White,Beige,None reported,Yes,240.0,2.0,4.0,9
1,188534,Land,Rover Defender SE,9142,Hybrid,8-Speed A/T,Silver,Black,None reported,Yes,395.0,3.0,6.0,4
2,188535,Ford,Expedition Limited,28121,Gasoline,10-Speed Automatic,White,Ebony,None reported,Yes,NaN,3.5,NaN,2
3,188536,Audi,A6 2.0T Sport,61258,Gasoline,Automatic,Silician Yellow,Black,None reported,Yes,NaN,NaN,NaN,8
4,188537,Audi,A6 2.0T Premium Plus,59000,Gasoline,A/T,Gray,Black,None reported,Yes,252.0,2.0,4.0,6


In [14]:
# Select numeric columns, excluding 'price' in train_data
numeric_cols = train_data.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = numeric_cols.drop('price')  # Exclude 'price' column from train_data

# Fill missing values for numeric columns in both train and test data
train_data[numeric_cols] = train_data[numeric_cols].fillna(train_data[numeric_cols].mean())
test_data[numeric_cols] = test_data[numeric_cols].fillna(test_data[numeric_cols].mean())


In [15]:

categorical_columns = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']


le = LabelEncoder()

for col in categorical_columns:
    train_data[col] = le.fit_transform(train_data[col].astype(str))
    test_data[col] = le.fit_transform(test_data[col].astype(str))  # Use fit_transform for consistency


In [30]:
# Create interaction features
train_data['milage_hp_ratio'] = train_data['milage'] / train_data['HP']
train_data['price_per_hp'] = train_data['price'] / train_data['HP']

# Apply the same transformations to test data
test_data['milage_hp_ratio'] = test_data['milage'] / test_data['HP']



In [31]:
train_data.head()

,id,brand,model,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,HP,Liters,Cylinders,car_age,milage_hp_ratio,price_per_hp
0,0,31,495,213000,2,38,312,71,1,0,4200,172.0,1.6,4.0,17,1238.372093,24.418605
1,1,28,930,143250,2,38,263,10,0,0,4999,252.0,3.9,8.0,22,568.452381,19.837302
2,2,9,1575,136731,1,38,38,71,1,0,13900,320.0,5.3,8.0,22,427.284375,43.437500
3,3,16,758,19500,2,49,29,14,1,0,45000,420.0,5.0,8.0,7,46.428571,107.142857
4,4,36,1077,7388,2,23,29,10,1,0,97500,208.0,2.0,4.0,3,35.519231,468.750000


In [32]:
# Remove outliers using IQR
Q1 = train_data['price'].quantile(0.25)
Q3 = train_data['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
train_data = train_data[(train_data['price'] >= lower_bound) & (train_data['price'] <= upper_bound)]


In [33]:
train_data.head()

,id,brand,model,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,HP,Liters,Cylinders,car_age,milage_hp_ratio,price_per_hp
0,0,31,495,213000,2,38,312,71,1,0,4200,172.0,1.6,4.0,17,1238.372093,24.418605
1,1,28,930,143250,2,38,263,10,0,0,4999,252.0,3.9,8.0,22,568.452381,19.837302
2,2,9,1575,136731,1,38,38,71,1,0,13900,320.0,5.3,8.0,22,427.284375,43.437500
3,3,16,758,19500,2,49,29,14,1,0,45000,420.0,5.0,8.0,7,46.428571,107.142857
5,5,3,182,40950,2,38,304,155,1,0,29950,252.0,2.0,4.0,6,162.500000,118.849206


In [40]:
X = train_data.drop(columns=['price','price_per_hp'])  # Features
y = train_data['price']  # Target

# For test data, we will use all the columns for prediction
X_test = test_data.copy()


In [41]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [42]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [43]:
y_pred_val = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f'Validation RMSE: {rmse}')


Validation RMSE: 12269.102771600583


In [38]:
# Check if X_train and X_test have the same columns
missing_in_test = set(X_train.columns) - set(X_test.columns)
extra_in_test = set(X_test.columns) - set(X_train.columns)

print(f'Missing in test: {missing_in_test}')
print(f'Extra in test: {extra_in_test}')


Missing in test: {'price_per_hp'}
Extra in test: set()


In [44]:
y_pred_test = model.predict(X_test)


In [45]:
y_pred_test

array([15708.61, 53837.63, 51283.36, ..., 22994.42, 15207.67, 35261.05])

In [46]:
submission = pd.DataFrame({
    'id': test_data['id'],
    'price': y_pred_test
})

submission.to_csv('submission.csv', index=False)


In [47]:
from xgboost import XGBRegressor

# XGBoost model
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Predicting
y_pred_xgb = xgb_model.predict(X_val)

# Calculate RMSE for XGBoost
rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_xgb))
print(f'XGBoost Validation RMSE: {rmse_xgb}')


XGBoost Validation RMSE: 12013.216481140787


In [ ]:
from lightgbm import LGBMRegressor

# LightGBM model
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
lgbm_model.fit(X_train, y_train)

# Predicting
y_pred_lgbm = lgbm_model.predict(X_val)

# Calculate RMSE for LightGBM
rmse_lgbm = np.sqrt(mean_squared_error(y_val, y_pred_lgbm))
print(f'LightGBM Validation RMSE: {rmse_lgbm}')


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006977 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1789
[LightGBM] [Info] Number of data points in the train set: 139478, number of used features: 14
[LightGBM] [Info] Start training from score 32272.652397


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Gradient Boosting model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Predicting
y_pred_gb = gb_model.predict(X_val)

# Calculate RMSE for Gradient Boosting
rmse_gb = np.sqrt(mean_squared_error(y_val, y_pred_gb))
print(f'Gradient Boosting Validation RMSE: {rmse_gb}')


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters grid
param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.05],
    'num_leaves': [31, 50, 100],
    'max_depth': [-1, 10, 20],
    'min_child_samples': [20, 30, 50],
    'subsample': [0.8, 0.9, 1.0]
}

# Instantiate the LGBMRegressor
lgbm = LGBMRegressor(random_state=42)

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters and RMSE after Grid Search
best_lgbm = grid_search.best_estimator_
y_pred_best_lgbm = best_lgbm.predict(X_val)
rmse_best_lgbm = np.sqrt(mean_squared_error(y_val, y_pred_best_lgbm))
print(f'Best LightGBM Validation RMSE: {rmse_best_lgbm}')


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
